In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://pypi.services.global.avant.com/simple/
     |████████████████████████████    | 277.3 MB 159.3 MB/s eta 0:00:01

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 317.0 MB 56 kB/s 
     |████████████████████████████████| 200 kB 100.5 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488511 sha256=dcfddd85c898e90e7a371184788ef457d8428b3d8b89c959c45b52e8585aa3ca
  Stored in directory: /home/jovyan/.cache/pip/wheels/92/09/11/aa01d01a7f005fda8a66ad71d2be7f8aa341bddafb27eee3c7
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession \
    .builder \
    .appName("genome_data_read") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/27 16:26:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark

In [15]:
df = spark.read.csv('sample_genome_data.csv',header = True)

In [16]:
df.show()

+------+----------+-------------+-----------+--------------------+--------+-----------+
|GeneID|Chromosome|StartPosition|EndPosition|            Sequence|GeneName|VariantType|
+------+----------+-------------+-----------+--------------------+--------+-----------+
|     1|         4|        38398|      38498|AGTTCAGGTCTAGTGAT...| GName_1|        SNP|
|     2|        11|        49330|      49430|AGCTCGGCAAGACGAGT...| GName_2|  Insertion|
|     3|        17|        49355|      49455|GGTTGGTCACGGTCGAA...| GName_3|  Insertion|
|     4|        15|        49699|      49799|ATCCGTCGTCGTCGGTC...| GName_4|        SNP|
|     5|        19|        34128|      34228|GTATTGGGCTCACACGA...| GName_5|  Insertion|
+------+----------+-------------+-----------+--------------------+--------+-----------+



In [ ]:
create table genome_date
(
    gene_id int increment() pk,
    chromosome int,
    start_position bigint,
    end_position bigint,
    genome_sequence string/varchar,
    gene_name string,
    variant_type string
)
USING 'S3:////'

    

In [40]:
from pyspark.sql.functions import col,length
from pyspark.sql.functions import udf 
from pyspark.sql.types import IntegerType 

In [34]:
'ATCG'.count('A')

1

In [35]:
def count_function(string):
    count_of_ATCG = string.count('ATCG')
    return count_of_ATCG
    

In [41]:
pyspark_udf = udf(count_function,IntegerType())

In [49]:
sequenct_count = df.select(["*",pyspark_udf("Sequence").alias("sequence_count"]))

In [50]:
df.show()

+------+----------+-------------+-----------+--------------------+--------+-----------+
|GeneID|Chromosome|StartPosition|EndPosition|            Sequence|GeneName|VariantType|
+------+----------+-------------+-----------+--------------------+--------+-----------+
|     1|         4|        38398|      38498|AGTTCAGGTCTAGTGAT...| GName_1|        SNP|
|     2|        11|        49330|      49430|AGCTCGGCAAGACGAGT...| GName_2|  Insertion|
|     3|        17|        49355|      49455|GGTTGGTCACGGTCGAA...| GName_3|  Insertion|
|     4|        15|        49699|      49799|ATCCGTCGTCGTCGGTC...| GName_4|        SNP|
|     5|        19|        34128|      34228|GTATTGGGCTCACACGA...| GName_5|  Insertion|
+------+----------+-------------+-----------+--------------------+--------+-----------+



In [52]:
final_df = df.concat(sequenct_count)

AttributeError: 'DataFrame' object has no attribute 'concat'

In [39]:
df.withColumn(col("Sequence")).rdd.map(count_function).alias(col('sequence_count'))

TypeError: withColumn() missing 1 required positional argument: 'col'